In [66]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np
import tensorflow as tf

true_stand_data=np.load(r'F:\\深度学习模型项目\\神经网络GRU模型\\data3\\data3//true_array.npy')
false_stand_data=np.load(r'F:\\深度学习模型项目\\神经网络GRU模型\\data3\\data3//false_array.npy')




In [67]:
true_stand_data.shape

(25230, 110, 7, 3)

In [88]:
true_stand_data=true_stand_data[:,:,:,0]
false_stand_data=false_stand_data[:,:,:,0]

IndexError: too many indices for array

In [92]:
true_stand_data=true_stand_data.swapaxes(1, 2)
false_stand_data=false_stand_data.swapaxes(1, 2)

In [94]:
false_stand_data.shape

(169426, 7, 110)

In [96]:
sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }

In [105]:
true_sample_channel_1 = []
false_sample_channel_1 = []


for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:]))
    
    
for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:]))
    
    

In [106]:
combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)


combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()

In [109]:
# Model 1
in_1 = Input(shape=(7,110))
x_480_1 = GRU(units=480,return_sequences=True,input_shape=(7,110))(in_1)
x_100_1 = GRU(units=100)(x_480_1)
x_10_1 =Dense(10, activation='tanh')(x_100_1)

model_final_dense_out = Dense(2, activation='softmax')(x_10_1)

model = Model(inputs=in_1, outputs=model_final_dense_out)
print(model.summary())


Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 7, 110)]          0         
_________________________________________________________________
gru_28 (GRU)                 (None, 7, 48)             23040     
_________________________________________________________________
gru_29 (GRU)                 (None, 20)                4200      
_________________________________________________________________
dense_18 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 22        
Total params: 27,472
Trainable params: 27,472
Non-trainable params: 0
_________________________________________________________________
None


In [110]:
import os
if 'GRU_predict_v8.h5' in os.listdir('./'):
    model = load_model("./GRU_predict_v8.h5")

callbacks_list = [    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=30
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict_v8.h5', 
    monitor='loss', 
    save_best_only=True),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.8,
        patience=5, 
        mode='auto',
        min_lr=0.00003)
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy']
             
             )

model.fit(x=[combin_train_data1,combin_train_data2,combin_train_data3],
          y=combin_target_data,
          batch_size=36,
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list      
         )


Epoch 1/100
5408/5408 [==============================] - 22s 4ms/step - loss: 0.6279 - accuracy: 0.6842
Epoch 2/100
5408/5408 [==============================] - 22s 4ms/step - loss: 0.5959 - accuracy: 0.7177
Epoch 3/100
5408/5408 [==============================] - 22s 4ms/step - loss: 0.5788 - accuracy: 0.7297
Epoch 4/100
5408/5408 [==============================] - 23s 4ms/step - loss: 0.5669 - accuracy: 0.7373
Epoch 5/100
5408/5408 [==============================] - 24s 4ms/step - loss: 0.5565 - accuracy: 0.7426
Epoch 6/100
5408/5408 [==============================] - 24s 4ms/step - loss: 0.5478 - accuracy: 0.7445
Epoch 7/100
5408/5408 [==============================] - 24s 4ms/step - loss: 0.5399 - accuracy: 0.7460
Epoch 8/100
5408/5408 [==============================] - 24s 4ms/step - loss: 0.5317 - accuracy: 0.7451
Epoch 9/100
5408/5408 [==============================] - 25s 5ms/step - loss: 0.5254 - accuracy: 0.7469
Epoch 10/100
5408/5408 [==============================] - 23s 4m

KeyboardInterrupt: 

In [12]:
b=true_stand_data[0].reshape(-1,1,11)

In [13]:
b.shape

(110, 1, 11)

In [6]:
# Model 1
in_1 = Input(shape=(110,1))
x_480_1 = GRU(units=480,return_sequences=True,input_shape=(110,1))(in_1)
x_100_1 = GRU(units=100)(x_480_1)
x_10_1 =Dense(10, activation='tanh')(x_100_1)

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np
import tensorflow as tf

true_stand_data=np.load(r'.//data3//true_array.npy')
false_stand_data=np.load(r'.//data3//false_array.npy')

true_stand_data=true_stand_data[:-100,:,:,:]
false_stand_data=false_stand_data[:-100,:,:,:]


true_stand_data=true_stand_data[:,:,:,0]
false_stand_data=false_stand_data[:,:,:,0]

true_stand_data=true_stand_data.swapaxes(1, 2)
false_stand_data=false_stand_data.swapaxes(1, 2)

sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }

true_sample_channel_1 = []
false_sample_channel_1 = []


for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:]))
    
    
for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:]))
    

combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)

combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()

# Model 1
in_1 = Input(shape=(7,110))
x_480_1 = GRU(units=480,return_sequences=True,input_shape=(7,110))(in_1)
x_480_1 = Dropout(0.2)(x_480_1)
x_100_1 = GRU(units=100)(x_480_1)
x_100_1 = Dropout(0.2)(x_100_1)
x_10_1 =Dense(10, activation='tanh')(x_100_1)

model_final_dense_out = Dense(2, activation='softmax')(x_10_1)

model = Model(inputs=in_1, outputs=model_final_dense_out)
print(model.summary())


import os
if 'GRU_predict_v8.h5' in os.listdir('./'):
    model = load_model("./GRU_predict_v8.h5")

callbacks_list = [    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=20
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict_v8.h5', 
    monitor='loss', 
    save_best_only=True),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.9,
        patience=10, 
        mode='auto',
        min_lr=0.00003)
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy']
             
             )

model.fit(x=combin_train_data1,
          y=combin_target_data,
          batch_size=360,
          epochs=300,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list,
          validation_split=0.2
         )




